### Definition Generation Evaluation

In [1]:
import gc
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import json
# model = "llama-2"
model = "vicuna"

# Load the JSON data
with open(f'drive/My Drive/PhD/LexDrafter_Paper/dataset/final_{model}_dataset.json', 'r') as json_file:
    data = json.load(json_file)

In [24]:
# found_records = [item for item in data if item.get("term") == "drive with sinusoidal input current"]
# found_records

[{'term': 'drive with sinusoidal input current',
  'celex_id': '32019R1781',
  'original_definition': 'drive with sinusoidal input current means a VSD with a sinusoidal waveform of the input current, characterised by a Total Harmonic Content below 10 %;',
  'generated_definition': "'drive with sinusoidal input current' means a VSD with a sinusoidal waveform of the input current, characterized by a Total Harmonic Content below 10%.",
  'existing_sentences': {'Article 3': ['‘drive with sinusoidal input current’ means a VSD with a sinusoidal waveform of the input current, characterised by a Total Harmonic Content below 10 %;']},
  'existing_record': ['NEW TERM'],
  'scores': {'Article 3': {'article_score': 1, 'statement_scores': [1]}}}]

#### Dataset creation

In [5]:
# Examples:
# predictions = ["hello there general kenobi", "foo bar foobar"]
# references = [
#    ["hello there general kenobi", "hello there !"],
#    ["foo bar foobar"]
# ]
# Initialize lists to store predictions and references
predictions = []
references = []
# Iterate through each item in the JSON data
for item in data:
  celex_id = item["celex_id"]
  if item["existing_record"] == ["NEW TERM"]:
    original_text = item['original_definition']
    term = item['term']
    new_term = f"'{term}'"
    original_text = original_text.replace(term, new_term)
    generated_text = item['generated_definition']

    if ((generated_text != "") and (generated_text != "NO JSON AS AN OUTPUT OBTAINED")):
      # Add the generated text to predictions list
      predictions.append(generated_text)

      # Add the original text to references list
      references_item = [original_text]  # References for this item
      references.append(references_item)
    # else:
    #   print(f"term is {term} for celex_id: {celex_id}")
    #   input("Press Enter to continue to the next iteration...")

In [7]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.1 MB/s eta 0:00:00


#### BLEU Score
- Reason for choosing
  
  - Length of the generated definitions matters
  - Precise definition is required


In [8]:
import evaluate
bleu = evaluate.load("bleu")

In [9]:
# Compute the BLEU score
bleu4_results = bleu.compute(predictions=predictions, references=references, max_order=4)
print(f"BLEU-4: {bleu4_results['bleu']}")

BLEU-4: 0.1837570668669643


In [10]:
# Compute the BLEU score
bleu3_results = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {bleu3_results['bleu']}")

BLEU-3: 0.21463045062072725


In [11]:
# Compute the BLEU score
blue2_results = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {blue2_results['bleu']}")

BLEU-2: 0.2602939269884561


In [12]:
# Compute the BLEU score
blue1_results = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {blue1_results['bleu']}")

BLEU-1: 0.35012966462552014


#### BERTScore

In [13]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.0 MB/s eta 0:00:00


In [14]:
import evaluate
bertscore = evaluate.load("bertscore")

In [15]:
bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

In [16]:
# Extract the F1 scores
f1_scores = bertscore_results['f1']

# Calculate the mean of F1 scores
mean_f1 = sum(f1_scores) / len(f1_scores)

In [17]:
print(f"BERTScore: {bertscore_results['f1']}")
print("--------")
print(f"Mean BERTScore: {mean_f1}")

BERTScore: [0.7893481254577637, 0.8395156264305115, 0.7065520286560059, 0.9032556414604187, 0.7933623194694519, 0.8514114618301392, 0.8437336683273315, 0.8123737573623657, 0.869597315788269, 0.8514766097068787, 0.8150394558906555, 0.8129146099090576, 0.8722251653671265, 0.7925906181335449, 0.8105833530426025, 0.9143986701965332, 0.8894253969192505, 0.7788251638412476, 0.8122238516807556, 0.9918109178543091, 0.9909533858299255, 0.981783390045166, 0.9292769432067871, 0.7836194634437561, 0.798564612865448, 0.7462249994277954, 0.855783224105835, 0.8597813248634338, 0.8340144753456116, 0.9780644178390503, 0.9619945883750916, 0.8170099854469299, 0.9654687643051147, 0.819178581237793, 0.7479830384254456, 0.7646585702896118, 0.802691638469696, 0.8391741514205933, 0.8350458741188049, 0.7831259369850159, 0.7674673795700073, 0.74443119764328, 0.8301801681518555, 0.8558171987533569, 0.7622718214988708, 0.7788939476013184, 0.773999810218811, 0.8869292140007019, 0.8441084623336792, 0.740773797035217

#### (NOT USED) BERTScore

In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets
from datasets import load_metric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00


In [ ]:
bertscore_metric = load_metric('bertscore')

<ipython-input-3-9b26049158da>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bertscore_metric = load_metric('bertscore')


In [ ]:
def bertscore_evaluation(generated_text, actual_text):
  bert_scores = bertscore_metric.compute(predictions=[generated_text], references=[actual_text], lang="en")
  return bert_scores['f1']

In [ ]:
original_text = """
"import duty" means customs duty payable on the import of goods;"""



vicuna_text = """
Import duty is a tax or fee charged by a country on goods that are imported into that country. It is typically imposed to raise revenue for the government or to protect domestic industries from foreign competition.
"""

In [ ]:
bertscore_value = bertscore_evaluation(vicuna_text, original_text)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bertscore_value

[0.8773407340049744]

#### (NOT USED) ROUGE
- Selected: https://github.com/danieldeutsch/sacrerouge/tree/master
- Refering to the paper: https://aclanthology.org/2023.acl-long.107.pdf

In [ ]:
!pip install sacrerouge==0.2.5

In [ ]:
!pip install urllib3==1.25.10
!apt install libxml-dom-perl

In [ ]:
!sacrerouge setup-metric rouge

In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [ ]:
from sacrerouge.metrics import Rouge

def rouge_evaluation(generated_text, actual_text):
  rouge = Rouge(max_ngram=4)
  scores = rouge.score(generated_text, [actual_text])
  return scores

In [ ]:
original_text = """
"import duty" means customs duty payable on the import of goods;"""

In [ ]:
values = rouge_evaluation(original_text, original_text)
values